# `Action Game Topic Analysis & Recommandation System`

If you find this helpful, please consider sharing it and giving it an upvote! Your support is appreciated.

![Action Game Analysis & Recommandations ](https://assets-prd.ignimgs.com/2022/08/10/top10fighting-1660101473258.jpg)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
import random


In [2]:
# Read the data file and adding back Popularity
df = pd.read_csv("/kaggle/input/imdb-video-games-dataset/imdb_video_games.csv", index_col=0)
df = df.rename(columns={"Unnamed: 0": "Popularity"})

# Data preprocessing and engineering

In [3]:
df = df[pd.to_numeric(df['User Rating'], errors='coerce').notna()]
df['Summary'].fillna('', inplace=True)  # Replace NaN with empty strings
df['Stars'].fillna('', inplace=True)  # Replace NaN with empty strings

df['User Rating'] = df['User Rating'].astype(float)
df.head()

,Title,Genre,User Rating,Number of Votes,Runtime,Year,Summary,Director,Stars,Certificate
Popularity,,,,,,,,,,
0,Baldur's Gate III,"Action, Adventure, Fantasy",9.8,"1,130",NaN,2023.0,Return to the Faerun in a tale of fellowship a...,"Swen Vincke,","Amelia Tyler, Neil Newbon, Jennifer English, D...",M
1,Starfield,"Action, Adventure, Drama",8.7,524,NaN,2023.0,"Set 300 years in the future, the player will a...","Todd Howard,","Sumalee Montano, Damien C. Haas, Barry Wiggins...",M
2,Red Dead Redemption II,"Action, Adventure, Crime",9.7,"47,844",NaN,2018.0,Amidst the decline of the Wild West at the tur...,NaN,"Benjamin Byron Davis, Roger Clark, Rob Wiethof...",M
3,Armored Core VI: Fires of Rubicon,"Action, Adventure, Sci-Fi",7.9,114,NaN,2023.0,Fifty years after a cataclysm caused by a subs...,"Masaru Yamamura, Kenneth Kin Yue Chan, Hidetak...","Fairouz Ai, Takayuki Sakazume, Megumi Han, Aya...",T
4,The Quarry,"Action, Adventure, Drama",8.0,"4,625",NaN,2022.0,When the sun goes down on the last night of su...,"Will Byles,","Siobhan Williams, Skyler Gisondo, Ted Raimi, G...",M


# # # # # # #   LSA****

In [4]:
tfidf = TfidfVectorizer(token_pattern=r'[a-zA-Z0-9]+', lowercase=True)
tfidf_matrix = tfidf.fit_transform(df['Summary'])


In [5]:
num_topics = 20  # 设置主题数
lsa = TruncatedSVD(n_components=num_topics, random_state=42)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

In [6]:
print("LSA Topic-Word Matrix:")
print(lsa.components_)

LSA Topic-Word Matrix:
[[ 1.13405003e-03  2.71905465e-03  1.10882480e-03 ...  4.27678229e-04
   1.92257361e-04  5.79641865e-04]
 [ 2.30643502e-03 -1.87480838e-03  3.26859600e-03 ... -1.62931698e-04
  -3.45074278e-04  9.23550379e-04]
 [ 4.71350951e-04  9.68707290e-04 -1.80033522e-03 ... -1.58215907e-03
   5.83365540e-05 -4.96537126e-04]
 ...
 [ 1.78749050e-03  1.48571534e-03  4.48789399e-04 ...  1.98870566e-03
   1.90233412e-03 -2.62734686e-04]
 [-1.65750992e-03 -7.61333781e-03  1.59462836e-03 ... -8.67312608e-05
   5.64351045e-04  5.86206928e-05]
 [ 4.28303708e-03  1.66976181e-04  1.13526523e-03 ... -2.29504559e-03
   8.76981441e-04 -1.63562660e-04]]


In [7]:
# 随机选择并打印主题中的一些单词
num_features = len(tfidf.get_feature_names_out())

for _ in range(5):
    random_word_index = random.randint(0, num_features - 1)
    random_word = tfidf.get_feature_names_out()[random_word_index]
    print("Random Word:", random_word)

Random Word: autonomous
Random Word: crowe
Random Word: aurelia
Random Word: whole
Random Word: philosopher


In [8]:
topic_index = 0
top_word_indices = lsa.components_[topic_index].argsort()[-5:]
top_words = [tfidf.get_feature_names_out()[index] for index in top_word_indices]
print("Top Words in Topic 0:", top_words)

Top Words in Topic 0: ['to', 'and', 'a', 'of', 'the']


In [9]:
for index in range(num_topics):
    top_word_indices = lsa.components_[index].argsort()[-20:]
    top_words = [tfidf.get_feature_names_out()[i] for i in top_word_indices]
    print(f'\nThe top 20 keywords in topic {index}:')
    print(top_words)


The top 20 keywords in topic 0:
['your', 'by', 'world', 'an', 'for', 'from', 's', 'with', 'his', 'as', 'on', 'you', 'game', 'is', 'in', 'to', 'and', 'a', 'of', 'the']

The top 20 keywords in topic 1:
['scrolling', 'in', 'this', 'arcade', 'developed', 'shooter', 'released', 'is', 'playstation', 'first', 'movie', 'fighting', 'film', 'for', 'series', 'action', 'on', 'based', 'video', 'game']

The top 20 keywords in topic 2:
['planet', 'original', 'star', 'same', 'follows', 'story', 'empire', 'following', 'movie', 'during', 'film', 'war', 'remake', 'series', 'version', 'years', 'after', 'events', 'of', 'the']

The top 20 keywords in topic 3:
['choose', 'way', 'all', 'more', 'will', 'battle', 'characters', 'are', 'fight', 'through', 'this', 'in', 'or', 'can', 'of', 'play', 'the', 'as', 'your', 'you']

The top 20 keywords in topic 4:
['has', 'must', 'stop', 'evil', 'film', 'movie', 'from', 'save', 'it', 'up', 'video', 'and', 's', 'on', 'the', 'game', 'his', 'based', 'sequel', 'to']

The top

In [10]:
df["Topic"] = lsa_matrix.argmax(axis=1)

In [11]:
# 保持数据处理方法不变
# 连接“Genre”和“Summary”列，用空格隔开
df2 = df.copy()
df2['Combined'] = df['Title'] + ', ' + df['Summary'] + ', ' + df['Genre']

In [12]:
print(df2['Combined'].head())

Popularity
0    Baldur's Gate III, Return to the Faerun in a t...
1    Starfield, Set 300 years in the future, the pl...
2    Red Dead Redemption II, Amidst the decline of ...
3    Armored Core VI: Fires of Rubicon, Fifty years...
4    The Quarry, When the sun goes down on the last...
Name: Combined, dtype: object


In [13]:
tfidf2 = TfidfVectorizer(token_pattern=r'[a-zA-Z0-9]+', lowercase=True)
tfidf_matrix2 = tfidf2.fit_transform(df2['Combined'])


In [14]:
print("Feature Names:", tfidf2.get_feature_names_out())
print("TF-IDF Matrix:")
print(tfidf_matrix2.toarray())
cosine_sim2 = cosine_similarity(tfidf_matrix2, tfidf_matrix2)


Feature Names: ['0' '000' '007' ... 'zwei' 'zx' 'zybex']
TF-IDF Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
def recommend(game, num_recommendations=10):
    # 找到DataFrame中输入游戏的索引
    game_index = df2[df2['Title'] == game].index
    if not game_index.empty:
        game_index = game_index[0]
    else:
        return "数据集中找不到该游戏。"

    # 计算输入游戏的余弦相似度分数
    cosine_scores = list(enumerate(cosine_sim2[game_index]))

    # 按相似度分数降序排序游戏
    cosine_scores = sorted(cosine_scores, key=lambda x: x[1], reverse=True)

    # 排除输入游戏（它将与自己具有相似度分数1）
    cosine_scores = cosine_scores[1:]

    # 获取前N个相似游戏的索引
    top_indices = [x[0] for x in cosine_scores[:num_recommendations]]

    # 返回包含推荐游戏详细信息的DataFrame
    recommended_games = df2.iloc[top_indices][['Title', 'Year', 'Genre', 'Certificate', 'User Rating']]
    
    return recommended_games

In [16]:
user_input1 = "Spider-Man"
recommendations1 = recommend(user_input1)

if isinstance(recommendations1, str):
    print(recommendations1)
else:
    print(f"对于 '{user_input1}' 的推荐游戏:")
    print(recommendations1)

对于 'Spider-Man' 的推荐游戏:
                              Title    Year                     Genre  \
Popularity                                                              
4326                    King Arthur  2004.0  Action, Adventure, Drama   
14246                    Crime City  1989.0             Action, Crime   
142         Crime Boss: Rockay City  2023.0     Action, Comedy, Crime   
10272             River City Ransom  1989.0  Action, Adventure, Crime   
15              Red Dead Redemption  2010.0  Action, Adventure, Crime   
903                The Godfather II  2009.0      Action, Crime, Drama   
3235                  Stranded Deep  2015.0         Action, Adventure   
360                    The Punisher  2004.0  Action, Adventure, Crime   
2            Red Dead Redemption II  2018.0  Action, Adventure, Crime   
13058                  Crime Cities  2001.0     Action, Crime, Sci-Fi   

           Certificate  User Rating  
Popularity                           
4326                 T  

# Analysis:

# Topic modeling using Latent Dirichlet Allocation (LDA) 

In [17]:
# CountVectorizer counts the frequency of words in a document without considering the importance of each word.
# # convert a collection of text documents in the 'Summary' column of the DataFrame df into a document-term matrix (DTM)
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

# creating a document-term matrix where each row represents a document (text summary) and
# each column represents a unique word (feature) in the text.
dtm = cv.fit_transform(df['Summary'])
print(dtm)

  (0, 6873)	1
  (0, 8176)	1
  (0, 3201)	1
  (0, 962)	1
  (0, 4998)	1
  (0, 210)	1
  (0, 6276)	1
  (0, 5530)	1
  (0, 201)	1
  (0, 754)	1
  (0, 4308)	1
  (0, 6824)	1
  (0, 8607)	1
  (0, 2160)	1
  (0, 2789)	1
  (0, 1931)	1
  (0, 8651)	1
  (0, 2979)	1
  (1, 7365)	1
  (1, 148)	1
  (1, 9255)	1
  (1, 3471)	1
  (1, 6181)	1
  (1, 675)	1
  (1, 7017)	1
  :	:
  (8652, 3060)	1
  (8652, 279)	1
  (8652, 593)	1
  (8652, 2135)	1
  (8652, 7051)	1
  (8652, 2740)	1
  (8652, 4672)	1
  (8652, 7519)	1
  (8652, 8538)	1
  (8652, 3183)	1
  (8652, 7755)	1
  (8652, 6653)	1
  (8652, 7322)	1
  (8652, 7976)	1
  (8652, 1052)	1
  (8652, 6849)	1
  (8652, 3460)	1
  (8652, 3075)	1
  (8652, 857)	1
  (8652, 6936)	2
  (8652, 2044)	1
  (8652, 1191)	1
  (8652, 321)	1
  (8652, 8052)	1
  (8652, 6455)	1


In [18]:
# This initializes an LDA model with 20 topics and a random state for reproducibility.
LDA = LatentDirichletAllocation(n_components=20,random_state=42)
LDA.fit(dtm)

LatentDirichletAllocation(n_components=20, random_state=42)

In [19]:
len(list(cv.get_feature_names_out()))

9317

## Exploring the vocabulary

In [20]:
# Explore the vocabulary by printing random words

num_features = len(cv.get_feature_names_out())

for _ in range(5):
    # Generate a random word index within the valid range
    random_word_index = random.randint(0, num_features - 1)
    
    # Get and print the random word from the vocabulary
    random_word = list(cv.get_feature_names_out())[random_word_index]
    print("Random Word:", random_word)



Random Word: voices
Random Word: predict
Random Word: prime
Random Word: cracking
Random Word: initial


## Top Words in a Single Topic:

In [21]:
# Get the word distribution for a specific topic (e.g., the first topic)
topic_index = 0
topic_word_distribution = LDA.components_[topic_index]

# Find the indices of the top 5 words in the topic by sorting based on word probabilities
top_word_indices = topic_word_distribution.argsort()[-5:]

# Retrieve and print the top words from the vocabulary
top_words = [list(cv.get_feature_names_out())[index] for index in top_word_indices]
print("Top Words in Topic:", top_words)


Top Words in Topic: ['known', 'player', 'team', 'alien', 'earth']


In [22]:
# Iterate through each topic in the LDA model
for index, topic in enumerate(LDA.components_):
    # Print the top 20 words for the current topic
    top_words = [list(cv.get_feature_names_out())[i] for i in topic.argsort()[-20:]]
    print(f'\nThe top 10 keywords in topic {index}:')
    print(top_words)



The top 10 keywords in topic 0:
['world', 'elite', 'forces', 'time', 'force', 'planet', 'mission', 'ship', 'sent', 'save', 'human', 'game', 'war', 'aliens', 'race', 'known', 'player', 'team', 'alien', 'earth']

The top 10 keywords in topic 1:
['players', 'death', 'missions', 'fight', 'story', 'paced', 'combat', 'weapons', 'platformer', 'levels', '2d', 'game', 'fast', 'set', 'scrolling', 'shooter', 'world', 'action', 'enemies', 'new']

The top 10 keywords in topic 2:
['enemy', 'family', 'taken', 'life', 'kill', 'facility', 'island', 'takes', 'terrorists', 'kidnapped', 'evil', 'time', 'player', 'mission', 'control', 'space', 'new', 'help', 'stop', 'rescue']

The top 10 keywords in topic 3:
['shooter', 'role', 'person', 'story', 'dragon', 'players', 'new', 'ii', 'different', 'wars', 'way', 'series', 'star', 'play', 'fight', 'set', 'player', 'war', 'game', 'world']

The top 10 keywords in topic 4:
['shooter', 'heroes', 'rpg', 'pokemon', 'magic', 'battle', 'set', 'time', 'defense', 'player

### Insights

Topic 6 looksing interesting, "vs", "street", "fighter"... it seems relating to people fighting on streets. Let's check it out if the legendary game Street Fighter falls into this category.

In [23]:
topics = LDA.transform(dtm)
df["Topic"]=topics.argmax(axis=1)

In [24]:
df[df.Title == "Street Fighter"]

,Title,Genre,User Rating,Number of Votes,Runtime,Year,Summary,Director,Stars,Certificate,Topic
Popularity,,,,,,,,,,,
1610,Street Fighter,Action,6.1,501,NaN,1987.0,"Two skilled martial arts experts, Ryu and Ken,...",NaN,Takashi Nishiyama,E10+,6


It is!!!

In [25]:
df.head()

,Title,Genre,User Rating,Number of Votes,Runtime,Year,Summary,Director,Stars,Certificate,Topic
Popularity,,,,,,,,,,,
0,Baldur's Gate III,"Action, Adventure, Fantasy",9.8,"1,130",NaN,2023.0,Return to the Faerun in a tale of fellowship a...,"Swen Vincke,","Amelia Tyler, Neil Newbon, Jennifer English, D...",M,1
1,Starfield,"Action, Adventure, Drama",8.7,524,NaN,2023.0,"Set 300 years in the future, the player will a...","Todd Howard,","Sumalee Montano, Damien C. Haas, Barry Wiggins...",M,16
2,Red Dead Redemption II,"Action, Adventure, Crime",9.7,"47,844",NaN,2018.0,Amidst the decline of the Wild West at the tur...,NaN,"Benjamin Byron Davis, Roger Clark, Rob Wiethof...",M,0
3,Armored Core VI: Fires of Rubicon,"Action, Adventure, Sci-Fi",7.9,114,NaN,2023.0,Fifty years after a cataclysm caused by a subs...,"Masaru Yamamura, Kenneth Kin Yue Chan, Hidetak...","Fairouz Ai, Takayuki Sakazume, Megumi Han, Aya...",T,3
4,The Quarry,"Action, Adventure, Drama",8.0,"4,625",NaN,2022.0,When the sun goes down on the last night of su...,"Will Byles,","Siobhan Williams, Skyler Gisondo, Ted Raimi, G...",M,10


# Recommendation System:

## Creating the TF-IDF Matrix

In [26]:
# Concatenate 'Genre' and 'Summary' columns with a space in between
df2 = df.copy()
df2['Combined'] = df['Title'] + ', ' + df['Summary'] + ', ' + df['Genre']

# Display the first few rows to verify the result
print(df2['Combined'].head())


Popularity
0    Baldur's Gate III, Return to the Faerun in a t...
1    Starfield, Set 300 years in the future, the pl...
2    Red Dead Redemption II, Amidst the decline of ...
3    Armored Core VI: Fires of Rubicon, Fifty years...
4    The Quarry, When the sun goes down on the last...
Name: Combined, dtype: object


In [27]:
# TfidfVectorizer is for Weighted Representation:
tfidf2 = TfidfVectorizer(token_pattern=r'[a-zA-Z0-9]+', lowercase=True)
tfidf_matrix2 = tfidf2.fit_transform(df2['Combined'])

# Show the TF-IDF features
print("Feature Names:", tfidf2.get_feature_names_out())
print("TF-IDF Matrix:")
print(tfidf_matrix2.toarray())

Feature Names: ['0' '000' '007' ... 'zwei' 'zx' 'zybex']
TF-IDF Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [28]:
cosine_sim2 = cosine_similarity(tfidf_matrix2, tfidf_matrix2)

In [29]:
def recommend(game, num_recommendations=10):
    # Find the index of the input game in the DataFrame
    game_index = df2[df2['Title'] == game].index
    if not game_index.empty:
        game_index = game_index[0]
    else:
        return "Game not found in the dataset."

    # Compute cosine similarity scores for the input game
    cosine_scores = list(enumerate(cosine_sim2[game_index]))

    # Sort the games by similarity score in descending order
    cosine_scores = sorted(cosine_scores, key=lambda x: x[1], reverse=True)

    # Exclude the input game (it will have a similarity score of 1 with itself)
    cosine_scores = cosine_scores[1:]

    # Get the indices of the top N similar games
    top_indices = [x[0] for x in cosine_scores[:num_recommendations]]

    # Return a DataFrame with the recommended games' details
    recommended_games = df2.iloc[top_indices][['Title', 'Year', 'Genre', 'Certificate', 'User Rating']]
    
    return recommended_games


In [30]:
user_input1 = "Grand Theft Auto V"
recommendations1 = recommend(user_input1)

# It checks if recommendations1 is a string (which would indicate an error message) using isinstance. 
# If it's a string, it prints the error message directly.
if isinstance(recommendations1, str):
    print(recommendations1)
else:
    print(f"Recommendations for '{user_input1}':")
    display(recommendations1)


Recommendations for 'Grand Theft Auto V':


,Title,Year,Genre,Certificate,User Rating
Popularity,,,,,
492,Grand Theft Auto 2,1999.0,"Action, Crime",M,7.5
610,Grand Theft Auto Online,2013.0,"Action, Crime, Drama",M,6.9
780,Grand Theft Auto: The Trilogy - The Definitive...,2021.0,"Action, Adventure, Comedy",M,4.0
653,Grand Theft Auto: Episodes from Liberty City,2009.0,"Action, Crime, Drama",M,8.4
1113,"Grand Theft Auto: London, 1969",1999.0,"Action, Crime",M,7.0
3533,Agent Cody Banks: Escape from Los Santos,2019.0,Action,NaN,6.5
1483,"Grand Theft Auto: London, 1961",1999.0,"Action, Crime",18,7.0
367,Grand Theft Auto,1997.0,"Action, Comedy, Crime",M,7.8
87,Grand Theft Auto III,2001.0,"Action, Crime, Drama",M,8.7


In [31]:
user_input2 = "Red Dead Redemption II"
recommendations2 = recommend(user_input2)

# It checks if recommendations2 is a string (which would indicate an error message) using isinstance. 
# If it's a string, it prints the error message directly.
if isinstance(recommendations2, str):
    print(recommendations2)
else:
    print(f"Recommendations for '{user_input2}':")
    display(recommendations2)


Recommendations for 'Red Dead Redemption II':


,Title,Year,Genre,Certificate,User Rating
Popularity,,,,,
15,Red Dead Redemption,2010.0,"Action, Adventure, Crime",M,9.4
3595,Dead or Alive 2,2000.0,"Action, Family, Mystery",T,7.2
1148,Red Dead Online,2019.0,"Action, Adventure, Crime",M,6.4
6159,"Wild, Wild West: The Steel Assassin",1999.0,"Action, Adventure, Mystery",T,7.0
4326,King Arthur,2004.0,"Action, Adventure, Drama",T,6.7
13055,Lead and Gold: Gangs of the Wild West,2010.0,"Action, Western",NaN,5.5
4487,Empire Earth II,2005.0,"Action, Adventure, Fantasy",T,6.4
3099,Operation Flashpoint: Red River,2011.0,"Action, War",M,6.5
147,Red Dead Revolver,2004.0,"Action, Adventure, Drama",M,7.8


`If you find this helpful, please consider sharing it and giving it an upvote! Your support is appreciated.`

# Other insightful datasets, and data science notebooks you may be interested:

| Dataset | URL |
| -- | -- |
|Top 240 Recommended Restaurants in L.A. 2023 | [link](https://www.kaggle.com/datasets/lorentzyeung/top-240-recommended-restaurants-in-la-2023)|
|Top 240 Recommended Restaurants in New York 2023 | [link](https://www.kaggle.com/datasets/lorentzyeung/top-240-recommended-restaurants-in-new-york-2023) |
|10 latest reviews of each London Mcdonald's | [link](https://www.kaggle.com/datasets/lorentzyeung/top-10-recent-comments-of-all-london-mcdonalds) |
|IMDb Action Video Games Dataset | [link](https://www.kaggle.com/datasets/lorentzyeung/imdb-video-games-dataset)  |
|Japanese Anime: An In-Depth IMDb Data Set | [link](https://www.kaggle.com/datasets/lorentzyeung/all-japanese-anime-titles-in-imdb)  |

--

| Notebook | URL |
| -- | -- |
|Sentiment Analysis for L.A. Restaurants 2023 | [link](https://www.kaggle.com/code/lorentzyeung/sentiment-analysis-for-l-a-restaurants-2023)|
|Sentiment Analysis for New York Restaurants 2023 | [link](https://www.kaggle.com/code/lorentzyeung/sentiment-analysis-for-new-york-restaurants-2023) |
|Social Listening for London Mcdonald's | [link](https://www.kaggle.com/code/lorentzyeung/social-listening-for-london-mcdonald-s) |
|Action Game Topic Analysis & Recommendation System | [link](https://www.kaggle.com/code/lorentzyeung/action-game-topic-analysis-recommendation-system)  |
|Anime Data EDA and Worldcloud | [link](https://www.kaggle.com/code/lorentzyeung/anime-data-eda-worldcloud)  |

Please don't forget to share it and giving it an upvote! Your support is appreciated.`